In [11]:
# pd.options.display.max_columns = None
# pd.options.display.max_rows = None

In [3]:
import pandas as pd
df = pd.read_excel("/users/amulya/Downloads/text_columns.xlsx")
# print(df.head())
print(df.dtypes)

Company                        object
Lists                          object
# Employees                   float64
Industry                       object
Website                        object
Company Linkedin Url           object
Company City                   object
Company State                  object
Company Country                object
Keywords                       object
SEO Description                object
Technologies                   object
Total Funding                 float64
Latest Funding                 object
Latest Funding Amount         float64
Last Raised At                 object
Annual Revenue                float64
Number of Retail Locations    float64
Short Description              object
Founded Year                  float64
ICP                            object
dtype: object


In [4]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import string
import contractions
from num2words import num2words
from textblob import TextBlob
from spellchecker import SpellChecker
from gensim.models import Word2Vec
import re
# from bs4 import BeautifulSoup
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

df = pd.read_excel("/users/amulya/Downloads/text_columns.xlsx")
a = df.isnull().sum()
# print(a)
text_columns = ['Keywords','SEO Description','Technologies','Short Description']
df['SEO Description'] = df['SEO Description'].fillna('others')
df['Keywords'] = df['Keywords'].fillna('others')
df['Technologies'] = df['Technologies'].fillna('others')
df['Short Description'] = df['Short Description'].fillna('others')


# print(df.head())
df['SEO Description'] = df['SEO Description'].str.lower()
df['SEO Description'] = df['SEO Description'].apply(word_tokenize)
punctuation = set(string.punctuation)
df['SEO Description'] = df['SEO Description'].apply(lambda tokens: [token for token in tokens if token not in punctuation])
stop_words = set(stopwords.words('english'))
df['SEO Description'] = df['SEO Description'].apply(lambda tokens: [token for token in tokens if token not in stop_words])
lemmatizer = WordNetLemmatizer()
df['SEO Description'] = df['SEO Description'].apply(lambda tokens: [lemmatizer.lemmatize(token) for token in tokens])
df['SEO Description'] = df['SEO Description'].apply(lambda tokens: [num2words(token) if token.isdigit() else token for token in tokens])
df['SEO Description'] = df['SEO Description'].apply(lambda tokens: [contractions.fix(token) for token in tokens])
df['SEO Description'] = df['SEO Description'].apply(lambda tokens: [token for token in tokens if token.isalnum()])
df['SEO Description'] = df['SEO Description'].apply(lambda tokens: [token.strip() for token in tokens])
df['SEO Description'] = df['SEO Description'].apply(lambda tokens: [re.sub(r'http\S+', '', token) for token in tokens])
df['SEO Description'] = df['SEO Description'].apply(lambda tokens: [re.sub(r'[\[\](){}]', '', token) for token in tokens])

# df['SEO Description'] = df['SEO Description'].apply(lambda text: BeautifulSoup(text, "html.parser").get_text())
spell = SpellChecker()
for tokens in df['SEO Description']:
    for token in tokens:
        if token is None:
           pass
        else:
            corrected_token = spell.correction(token)
           
# Join tokens back to strings
df['SEO Description'] = df['SEO Description'].apply(' '.join)

print(df['SEO Description'])
w2v_model = Word2Vec(sentences=df['SEO Description'], vector_size=100, window=5, min_count=1, workers=4)
w2v_model.train(df['SEO Description'], total_examples=len(df['SEO Description']), epochs=10)
word_vectors = w2v_model.wv
print(w2v_model)
print('hdgvdbvb')

[nltk_data] Downloading package punkt to /Users/amulya/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/amulya/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/amulya/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


0       gainsight customer success product experience ...
1                                                  others
2       build web mobile apps faster laravel android f...
3       runo call management app manage call sim level...
4       accelerate ecommerce data analytics journey sa...
                              ...                        
1712                                    ordering platform
1713                                               others
1714                                               others
1715                                               others
1716                                               others
Name: SEO Description, Length: 1717, dtype: object
Word2Vec<vocab=55, vector_size=100, alpha=0.025>
hdgvdbvb


In [6]:
def preprocess_text(text):
    text = text.lower()
    tokens = word_tokenize(text)
    tokens = [token for token in tokens if token not in string.punctuation]
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    tokens = [num2words(token) if token.isdigit() else token for token in tokens]
    tokens = [contractions.fix(token) for token in tokens]
    tokens = [token for token in tokens if token.isalnum()]
    tokens = [token.strip() for token in tokens]
    tokens = [re.sub(r'http\S+', '', token) for token in tokens]
    spell = SpellChecker()
    corrected_tokens = []
    for token in tokens:
        corrected_token = spell.correction(token)
        corrected_tokens.append(corrected_token)

    return corrected_tokens
def train_word2vec_model(tokenized_text):
    word2vec_model = Word2Vec(sentences=tokenized_text, vector_size=100, window=5, min_count=1, workers=4)
    word2vec_model.train(tokenized_text, total_examples=len(tokenized_text), epochs=10)
    return word2vec_model

# Load the DataFrame
df = pd.read_excel("/users/amulya/Downloads/text_columns.xlsx")

# Define the text columns
text_columns = ['Keywords', 'SEO Description', 'Technologies', 'Short Description']
df['Keywords'] = df['Keywords'].astype(str)
df['SEO Description'] = df['SEO Description'].astype(str)
df['Technologies'] = df['Technologies'].astype(str)
df['Short Description'] = df['Short Description'].astype(str)
df['SEO Description'] = df['SEO Description'].fillna('others')
df['Keywords'] = df['Keywords'].fillna('others')
df['Technologies'] = df['Technologies'].fillna('others')
df['Short Description'] = df['Short Description'].fillna('others')
word2vec_models = {}

# Apply preprocessing to each text column
for col in text_columns:
    df[col] = df[col].apply(preprocess_text)
    tokenized_text = df[col].tolist()
    word2vec_model = train_word2vec_model(tokenized_text)
    word2vec_models[col] = word2vec_model
    # print(word2vec_model.wv.most_similar('example'))
    print('ikdjshbn')
print(df['Keywords'])

ikdjshbn
ikdjshbn
ikdjshbn


InvalidOperation: [<class 'decimal.ConversionSyntax'>]